# Using the TMDB API

In [121]:
import requests
import json
import gzip
import pandas as pd
import urllib.request

## Movie IDs DataFrame

We need to be able to look up the TMDB ID for each movie, so first of all we download, unzip and store this data in a data frame.

In [191]:
# Get the ID file from tmdb
tmdb_id_file = urllib.request.URLopener()
tmdb_id_file.retrieve("http://files.tmdb.org/p/exports/movie_ids_10_09_2021.json.gz", "./data/tmdb_id_file.gz")

# Unzip it and store the string in a list of strings
with gzip.GzipFile("./data/tmdb_id_file.gz", 'r') as fin:   
    json_bytes = fin.read()                      
json_list_of_str = json_bytes.decode().split("\n")

# Turn the list of strings into a list of dictionaries
dict_list = []
for dict_str in json_list_of_str[:-1]:
    real_dict = json.loads(dict_str)
    dict_list.append(real_dict)
    
# Convert it to a dataframe
movie_id_df = pd.DataFrame(dict_list)
movie_id_df.head()

,adult,id,original_title,popularity,video
0,False,3924,Blondie,1.185,False
1,False,6124,Der Mann ohne Namen,0.600,False
2,False,8773,L'amour à vingt ans,2.733,False
3,False,25449,New World Disorder 9: Never Enough,1.545,False
4,False,31975,Sesame Street: Elmo Loves You!,0.600,True


## Getting Info for individual movies

In [4]:
# load the API credentials
key_yml = json.load(open("./tmdb_credentials.yml"))
tmdb_api_key = key_yml["api_key"]

'0d0cadc24e57023358add396ceb15f3e'

In [45]:
# Here's an example URL for one specific movie - note the ID = 55
example_url = "https://api.themoviedb.org/3/movie/55?api_key="

In [183]:
req = requests.get(example_url + tmdb_api_key)

# Here's what the output looks like
example_json = req.json()
example_json

{'adult': False,
 'backdrop_path': '/3YzVtvNERQOIJXnrcWv3xhj5EMT.jpg',
 'belongs_to_collection': None,
 'budget': 2000000,
 'genres': [{'id': 18, 'name': 'Drama'}, {'id': 53, 'name': 'Thriller'}],
 'homepage': '',
 'id': 55,
 'imdb_id': 'tt0245712',
 'original_language': 'es',
 'original_title': 'Amores perros',
 'overview': 'A fatalistic car crash in Mexico City sets off a chain of events in the lives of three persons: a young man aching to run off with his sister-in-law, a supermodel, and a homeless man. Their lives are catapulted into unforeseen circumstances instigated by the seemingly inconsequential destiny of a dog.',
 'popularity': 22.476,
 'poster_path': '/vV4vlD4ool5JSsS1rB82qjCF6z8.jpg',
 'production_companies': [{'id': 5084,
   'logo_path': None,
   'name': 'Altavista Films',
   'origin_country': 'MX'},
  {'id': 110056,
   'logo_path': None,
   'name': 'Zeta Film (MX)',
   'origin_country': ''},
  {'id': 1632,
   'logo_path': '/cisLn1YAUuptXVBa0xjq7ST9cH0.png',
   'name': '

In [184]:
# We can get the description with 'overview'
example_json["overview"]

'A fatalistic car crash in Mexico City sets off a chain of events in the lives of three persons: a young man aching to run off with his sister-in-law, a supermodel, and a homeless man. Their lives are catapulted into unforeseen circumstances instigated by the seemingly inconsequential destiny of a dog.'